In [2]:
from scipy import optimize
import numpy as np
% matplotlib inline

### Deterministic Model

In [11]:
def deter_obj(x, q, c, e, f):
    """
        Objective function for deterministic problem. Single time horizon - no uncertainty.
        
        Parameters
        ----------
        x : {np.array}
            decision variables
        
        q : {np.array}
            sale price
        
        c : {np.array}
            purchase cost
        
        e : {np.array}
            inventory return
        
        f : {np.array}
            operation cost
    """
    s = x[:2]
    p = x[2:4]
    I = x[4:8]
    R = x[8:]
    
    return np.dot(q, s) - np.dot(c,p) + np.dot(e,I) - np.dot(f,R)

In [16]:
# sales price
q = np.array([100,80])

# purchase cost
c = np.array([10, 15])

# inventory coefficients
e = np.array([10, 15, 20, 25])

# operation cost
f = np.array([30, 35])

# decision variables - s3, s4, p1, p2, I1_f, I2_f, I3_f, I4_f, R1, R2

# bounds
bounds = [(20, 40), (20, 85), (20, 100), (30, 200), (0, 20), (0, 20), (0, 20), (0, 20), (20, 100), (30, 100)]

# init vals - boundary midpoints
x0 = np.array([(tup[1] - tup[0])/2 for tup in bounds])

cons = (
    {"type": "eq", "fun": lambda x: x[2] - 0.4*x[8] - x[4]},
    {"type": "eq", "fun": lambda x: x[3] - 0.6*x[8] - x[9] - x[5]},
    {"type": "eq", "fun": lambda x: 0.5*x[8] - x[6] - x[0]},
    {"type": "eq", "fun": lambda x: 0.6*x[9] - x[7] - x[1]}
)

res = optimize.minimize(deter_obj, x0, args=(q,c,e,f), bounds=bounds, constraints=cons)

if res.success:
    print "Num obj func evaluations: %s" % res.nfev
    print "Num iterations: %s" % res.nit

print res.message
print "\n"
    
x = res.x

print "Sales Units - %s" % ", ".join(["S%s: %s" % (idx, val) for idx, val in enumerate(x[:2])])
print "Purchase Units - %s" % ", ".join(["P%s: %s" % (idx, val) for idx, val in enumerate(x[2:4])])
print "Inventory Units - %s" % ", ".join(["I%s_final: %s" % (idx, val) for idx, val in enumerate(x[4:8])])
print "Operation Levels - %s" % ", ".join(["R%s: %s" % (idx, val) for idx, val in enumerate(x[8:])])

Num obj func evaluations: 84
Num iterations: 7
Optimization terminated successfully.


Sales Units - S0: 19.9999999999, S1: 40.0000000002
Purchase Units - P0: 41.0000000001, P1: 148.000000567
Inventory Units - I0_final: 8.99999999993, I1_final: 5.65646643253e-07, I2_final: 20.0000000002, I3_final: 20.0000000002
Operation Levels - R0: 80.0000000003, R1: 100.000000001


### Two Stage Stochastic Programming Model

### RO SP FR

### Mean Markovitz Model

In [6]:
def risk_const(x, params, risk_param, profit_lb):
    """
        Risk constraint for Mean-Markovitz model.
        
        Parameters
        ----------
        x : {np.array}
            array of decision variables
            
        params : {dict}
            dictionary of parameters used for constraint generation
        
        risk_param : {float}
            custom risk parameter for variation penalty
        
        profit_lb : {float}
            lower bound for profit (t)
            
        Parameter Dictionary
        --------------------
        sales_mean : {np.array}
            mean coefficients for sales variable (selling price)
        
        sales_std : {np.array}
            standard deviation for sales variables
            
        purchase_coeff : {np.array}
            purchase cost
        
        inventory_coeff : {np.array}
            positive inventory returns
        
        operation_coeff : {np.array}
            operation costs
    """
    # segment decision variables
    s = x[:2]
    p = x[2:4]
    I = x[4:8]
    R = x[8:]
    
    mu_q = params["sales_mean"]
    std_q = params["sales_std"]
    c = params["purchase_coeff"]
    e = params["inventory_coeff"]
    f = params["operation_coeff"]
    
    return np.dot(mu_q, s) - np.dot(c,p) + np.dot(e,I) - np.dot(f,R) - risk_param*np.sqrt(np.dot(np.square(std_q),np.square(y))) - profit_lb

In [ ]:
# profit lb
t = 10000

# risk param
theta = 0.3

# mean sales price
mean_q = np.array([90, 80])

# std sales price
std_q = np.array([27, 20])

# purchase cost
c = np.array([10, 15])

# inventory coefficients
e = np.array([10, 15, 20, 25])

# operation cost
f = np.array([30, 35])

# decision variables - s3, s4, p1, p2, I1_f, I2_f, I3_f, I4_f, R1, R2

# bounds
bounds = [(20, 40), (20, 85), (20, 100), (30, 200), (0, 20), (0, 20), (0, 20), (0, 20), (20, 100), (30, 100)]

# init vals - boundary midpoints
x0 = np.array([(tup[1] - tup[0])/2 for tup in bounds])

# maximize profit
obj = lambda x: -t

params = {"sales_mean":mean_q, "sales_std": std_q, "purchase_coeff": c, "inventory_coeff": e,
          "operation_coeff": f}

# constraints
cons = (
    {"type": "ineq", "fun": risk_const, "args": (params,theta,t)},
    {"type": "eq", "fun": lambda x: x[2] - 0.4*x[8] - x[4]},
    {"type": "eq", "fun": lambda x: x[3] - 0.6*x[8] - x[9] - x[5]},
    {"type": "eq", "fun": lambda x: 0.5*x[8] - x[6] - x[0]},
    {"type": "eq", "fun": lambda x: 0.6*x[9] - x[7] - x[1]}
)

res = optimize.minimize(obj, x0, bounds=bounds, constraints=cons)

x = res.x

print "Sales Units - %s" % ", ".join(["S%s: %s" % (idx, val) for idx, val in enumerate(x[:2])])
print "Purchase Units - %s" % ", ".join(["P%s: %s" % (idx, val) for idx, val in enumerate(x[2:4])])
print "Inventory Units - %s" % ", ".join(["I%s_final: %s" % (idx, val) for idx, val in enumerate(x[4:8])])
print "Operation Levels - %s" % ", ".join(["R%s: %s" % (idx, val) for idx, val in enumerate(x[8:])])